In [0]:
%run ./transform

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import sys
import os

from augmentation_config import augmentation_profiles

In [0]:
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

def count_bw_mask_pil(mask_path):
    arr = np.array(Image.open(mask_path).convert("L"))
    white = np.sum(arr == 255)
    black = np.sum(arr == 0)
    return black, white

# Samle data
black_counts = []
white_counts = []
filenames = []

folder = Path("/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/lab/")
for p in folder.glob("*.png"):
    black, white = count_bw_mask_pil(p)
    black_counts.append(black)
    white_counts.append(white)
    filenames.append(p.name)

# Lag en DataFrame
df = pd.DataFrame({
    "filename": filenames,
    "black_pixels": black_counts,
    "white_pixels": white_counts
})

# Beregn statistikk
summary = {
    "Total": df[["black_pixels", "white_pixels"]].sum(),
    "Mean": df[["black_pixels", "white_pixels"]].mean(),
    "Min": df[["black_pixels", "white_pixels"]].min(),
    "Max": df[["black_pixels", "white_pixels"]].max(),
    "Std": df[["black_pixels", "white_pixels"]].std()
}

stats_table = pd.DataFrame(summary).T

# Vis tabell
from IPython.display import display
styled_table = stats_table.style.format({
    "black_pixels": "{:.2e}",
    "white_pixels": "{:.2e}"
}).set_caption("Mask pixel statistics") \
  .set_table_styles([
      {"selector": "caption", "props": [("font-size", "14pt"), ("font-weight", "bold")]},
      {"selector": "th, td", "props": [("border", "1px solid gray"), ("padding", "8px")]}
  ]) \
  .set_properties(**{
      "text-align": "right",
      "border-collapse": "collapse"
  })

display(styled_table)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# Regn ut andel hvite piksler
proportions = [w / (w + b) for w, b in zip(white_counts, black_counts)]
mean_prop = np.mean(proportions)
median_prop = np.median(proportions)

plt.figure(figsize=(14, 6))
plt.bar(range(len(proportions)), proportions, label="Proportion per mask")
plt.axhline(mean_prop, color="red", linestyle=":", linewidth=2, label=f"Mean: {mean_prop:.2%}")
plt.axhline(median_prop, color="red", linestyle="--", linewidth=2, label=f"Median: {median_prop:.2%}")

plt.ylabel("Proportion of white pixels")
plt.xlabel("Mask index")
plt.title("Proportion of white pixels per mask")
plt.legend()
plt.tight_layout()
plt.show()


In [0]:
totals = stats_table.loc["Total"]

plt.figure(figsize=(6, 5))
bars = plt.bar(totals.index, totals.values, color=["black", "lightgray"])
plt.title("Total number of pixels across all masks")
plt.ylabel("Pixel count")

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        f"{height:,.0f}",  # tusenskille og ingen desimaler
        ha="center",
        va="bottom",
        fontsize=11,
        fontweight="bold"
    )

plt.tight_layout()
plt.show()

In [0]:
# Regn ut totaler (hvis du ikke har stats_table fra før)
total_black = df["black_pixels"].sum()
total_white = df["white_pixels"].sum()
total_pixels = total_black + total_white

# Vekter til Dice og BCE basert på pikselfordeling
dice_weight = total_white / total_pixels
bce_weight = total_black / total_pixels

# pos_weight til BCEWithLogitsLoss (for å kompensere for skjevhet)
pos_weight = total_black / total_white

print(f"Dice weight: {dice_weight:.4f}")
print(f"BCE weight:  {bce_weight:.4f}")
print(f"pos_weight (BCEWithLogitsLoss): {pos_weight:.2f}")


In [0]:
def interactive_visualize(image_dir, mask_dir, dom_dir):
    """
    Åpner et vindu der du kan bla i bilder, masker og DOM ved å trykke på piltastene.
    """
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".png")])
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))
    manager = getattr(fig.canvas, "manager", None)
    if manager is not None and hasattr(manager, "set_window_title"):
        manager.set_window_title("Trykk ⬅️ eller ➡️ for å bla")
    idx = [0]

    def show(i):
        image_path = os.path.join(image_dir, image_files[i])
        mask_path = os.path.join(mask_dir, image_files[i].replace("image", "mask"))
        dom_path = os.path.join(dom_dir, image_files[i].replace("image", "dom"))
        img = Image.open(image_path)
        mask = Image.open(mask_path)
        dom = Image.open(dom_path)

        ax[0].imshow(img)
        ax[0].set_title(f"Bilde: {image_files[i]}")
        ax[1].imshow(mask, cmap="gray")
        ax[1].set_title("Maske")
        ax[2].imshow(dom)
        ax[2].set_title("DOM")
        for a in ax:
            a.axis("off")
        fig.canvas.draw_idle()

    def on_key(event):
        if event.key == "right":
            idx[0] = (idx[0] + 1) % len(image_files)
            show(idx[0])
        elif event.key == "left":
            idx[0] = (idx[0] - 1) % len(image_files)
            show(idx[0])
        elif event.key == "escape":
            plt.close(fig)

    fig.canvas.mpl_connect("key_press_event", on_key)
    show(idx[0])
    plt.tight_layout()
    plt.show()


def visualize_multiple_augmentations(image_path, mask_path, cfg_name="basic", n=4):
    """Visualiserer flere augmentasjoner av et bilde og tilhørende maske."""
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path)) // 255
    transform = get_train_transforms(augmentation_profiles[cfg_name])

    fig, axes = plt.subplots(n, 2, figsize=(10, 5 * n))
    if n == 1:
        axes = [axes]

    for i in range(n):
        augmented = transform(image=image, mask=mask)
        image_aug = augmented["image"].permute(1, 2, 0).numpy()
        mask_aug = augmented["mask"].numpy()

        axes[i][0].imshow(image_aug)
        axes[i][0].set_title(f"Augmentert bilde {i+1}")
        axes[i][1].imshow(mask_aug, cmap="gray")
        axes[i][1].set_title(f"Maske {i+1}")

        for a in axes[i]:
            a.axis("off")

    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    visualize_multiple_augmentations(
        image_path="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/img/image_249322_6786313_249385_6786382.png",
        mask_path="/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/lab/mask_249322_6786313_249385_6786382.png",
        cfg_name="default",  # Se augmentation_config.py
    )